In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf
#from sklearn import metrics
import pickle
from lib import accuracy
from lib import labels

In [2]:
data_dir = "../data/"
pickle_file = 'tf_data_mean.pickle'

with open(data_dir + pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = np.matrix(save['train_data'], dtype='float32')
    train_labels = np.matrix(save['train_labels'], dtype='float32')
    valid_dataset = np.matrix(save['validate_data'], dtype='float32')
    valid_labels = np.matrix(save['validate_labels'], dtype='float32')
    test_dataset = np.matrix(save['test_data'], dtype='float32')
    test_bids = list(save['test_business_ids'])
    #test_bids = np.ravel(test_bids)
    del save  # hint to help gc free up memory
    print 'Training set', train_dataset.shape, train_labels.shape
    print 'Validation set', valid_dataset.shape, valid_labels.shape
    print 'Test set', test_dataset.shape, len(test_bids)

# Convert labels to a dict of binarized labels [1. if true, 1. if false]
# So can be used for softmax per label.
train_labels = labels.binarize_softmax_labels(train_labels)
valid_labels = labels.binarize_softmax_labels(valid_labels)

Training set (1600, 1024) (1600, 9)
Validation set (400, 1024) (400, 9)
Test set (10000, 1024) 10000


In [3]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.

#set to -1 to use all data
train_subset = 1600
input_size = 1024
labels_size = 2
label_to_train = 0
    
graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[label_to_train][:train_subset, :])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels[label_to_train])
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([input_size, labels_size]))
    #biases = tf.Variable(tf.zeros([labels_size]))
    biases = tf.Variable(tf.zeros([labels_size]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(
        #This uses sigmoid cross entropy, which allows for multiple labels
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 3000

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print 'Initialized'
    for step in xrange(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            #print predictions.shape
            print ""
            print ">---------- step:", step
            print 'Loss at step', step, ':', l
            #print "F1 Score:", (valid_labels)
            print "Training accuracy: {0:.0f}%".format(100 * accuracy.binarized_accuracy(
               train_labels[label_to_train][:train_subset, :], predictions))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print "Validation accuracy: {0:.0f}%".format(100 * accuracy.binarized_accuracy(
                 valid_labels[label_to_train], valid_prediction.eval()))
            
            test_results = test_prediction.eval()
            
#>---------- step: 4900
#Loss at step 4900 : 0.0220133
#Training accuracy: 100%
#Validation accuracy: 74%




In [ ]:
train_dataset = save['train_dataset']
train_labels = save['train_labels']
valid_dataset = save['valid_dataset']
valid_labels = save['valid_labels']
test_dataset = save['test_dataset']
test_bids = save['test_bids']
del save  # hint to help gc free up memory
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

In [ ]:
#print test_bids

test_results = np.array(np.round(test_results), dtype=bool)
output = "business_id,labels\n"

for i in range(len(test_bids)):
    print i
    tags = np.argwhere(test_results[i])
    tags_str = ' '.join('%d'%F for F in tags[:] )
    output += test_bids[i]+ "," + tags_str + "\n"
    
print output
f = open('test_submission', 'w')
f.write(output)
f.close()

In [ ]:
output

In [ ]:
test_dataset[1]